<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TSA_Nifty_50_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pending Activities** -

Titles and axes names for all graphs
check if auto_arima works here instead of your for loop

model using ARCH

# **# Group - 8**

CH24M548 - Mounika Chowdary Pamulapati

CH24M571 - Soumya Mukherjee

CH24M577 - Surya Kandala

The steps followed to model the given Nifty 50 data were the below:

1. **Importing the dataset**.
2. **Cleaning the dataset**: Ensured that missing dates were added and forward fill of the data for the missing dates. Dropped NaN records whereever observed.
3. **Stationartiy Check**: Validated if the timeseries is stationary or not by performing ADF and KSS test. For ADF test if p<0.05, then it says the timeseries is stationary. If p>0.05 for KSS test, it indicates the timeseries is stationary.
4. **First Order Differencing & Log Differencing**: Performing first order differencing to make the timeseries stationary.
5. **ACF and PACF**: Compute Auto Correlation and Partial correlation to understand the presence of MA and AR components.
6. **Power Spectral Density**: For a process to be stationary, both mean and variance need to be independent of the absolute time. Hence, validating if Variance is dependent on the absolute time.
7. **Random Walk**: Based on the ACF and PACF values, going with ARMA model of 010 since ACF and PACF results indicate complete white noise.
8. **Akaike's Information Criteria**: Since the forecast is only a straight line, checked the AIC values for models starting from 000 till 332. Chose that model which has less AIC value
9. **Train, Test & Generate Forecast**: Generated the forecast for 122 model which got less AIC value in step 7.
10. **MSE, RMSE, MAPE**: Identify the Mean Square Error and Root Mean Square Error to check the residual details.
11. **Alternate Non-Lionear models**: Since we have variance as a function of absolute time, linear models might not be the right choice to move ahead, hence trying out the non-linear models ARCH and GARCH to model the data.

Packages Required!

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install statsmodels
!pip install scipy
!pip install scikit-learn
!pip install arch
!pip install statsforecast
!pip install dask[dataframe]
!pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import signal
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model
from statsforecast import StatsForecast
import pmdarima as pm
import warnings
import scipy.stats as stats
from scipy.stats import boxcox

**Importing the dataset**:

Reading the Training dataset from Google Drive and set the date field as index.

In [ ]:
# Generate or load your dataset
url='https://drive.google.com/file/d/16h5NCff5s-2V9lwksc9aCEKJIAn2Znyv/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
# df = pd.read_csv('NIFTY_50_2015-2023.csv')
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df.set_index('Date', inplace=True)

df.head()

In [ ]:
# Import Test dataset

url='https://drive.google.com/file/d/1L-hcnQLVffdxz1PwwMTJkB6AmB2BkjBn/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_test = pd.read_csv(url)
## df_test = pd.read_csv('NIFTY_50_2024.csv')
df_test['Date'] = pd.to_datetime(df_test['Date'], format='mixed')
df_test['Date'] = pd.to_datetime(df_test['Date'])
df_test.set_index('Date', inplace=True)


df_test.head()

**Statistical Description** of Train and Test datasets

In [ ]:
# Statistical Description of training data

df.describe().T

In [ ]:
# Statistical Description of the test data

df_test.describe().T

In [ ]:
# checking if there are any outliers in the data using Box Plot

plt.figure(figsize=(12, 8))
plt.boxplot(df['Close'],patch_artist = True, vert = 0)
plt.title('Box Plot of Close Value')
plt.xlabel('Closing Value')
plt.show()


**Cleaning the dataset**:

Checking for missing dates and adding them back to the dataset.
Forwardfill the values from the previous rows. -- Currently, even if stock market is not open on Saturday and Sunday, still Friday closing will be the opening for Monday. So, assuming forward fill imputation would not be a problem.

In [ ]:
start = df.index[0].date() # minimum date from the dataset
end = df.index[len(df)-1].date() # maximum date from the dataset
new_dates = pd.date_range(start=start,end=end,freq='D') # Generating new dates starting from minimum and extending till maximum

# Re-indexing new dates with the old dates

df = df.reindex(new_dates)
df = df.rename_axis('Fin_Dt')
df.reset_index(inplace=True)
df = df.ffill()

# Maintaining one date column as index and other date column for our reference.

df['Date'] = pd.to_datetime(df['Fin_Dt'])
df.set_index('Date', inplace=True)

df.head()

In [ ]:
# Check for missing dates and add them to the dataset
# Identify Start and End dates from the dataset and generate new dates from those dates which will be daily dates

start = df_test.index[0].date()
end = df_test.index[len(df_test)-1].date()
new_dates = pd.date_range(start=start,end=end,freq='D')

# Re-indexing new dates with the old dates

df_test = df_test.reindex(new_dates)
df_test = df_test.rename_axis('Fin_Dt')
df_test.reset_index(inplace=True)
df_test = df_test.ffill()

df_test['Date'] = pd.to_datetime(df_test['Fin_Dt'])
df_test.set_index('Date', inplace=True)

df_test.head()

**Plot the Training & Test dataset**.

Graph is being plotted for the Close values as training and testing to be done for that column of the dataset.

In [ ]:
# Use the 'Close' prices as the target variable and plot the Original Data
final_data = df[['Fin_Dt','Close']] ## will be used at a later point in the code.

final_data_test = df_test[['Fin_Dt','Close']]

close_plot_test = final_data_test['Close']

plt.figure(figsize=(12, 6))
plt.plot(final_data['Close'], label='Training Data')
plt.plot(close_plot_test, color = 'red', label = 'Test Data')
plt.title("Nifty 50 - Training & Test Data")
plt.xlabel("Year")
plt.ylabel("Closing Value")
plt.legend()
plt.show()

**Stationarity Check**:

To check if the data is stationary, here we used Augmented Dickey Fuller's Test.
This test helps us understand if there is a change in the Mean with absolute time. But, it doesn't give any information on the variance of the timeseries.


In [ ]:
# Perform the ADF test
adf_res = adfuller(df['Close'], autolag='AIC')

print(f"ADF Statistic: {adf_res[0]}")
print(f"p-value: {adf_res[1]}")
print(f"No Lags Used: {adf_res[2]}")
print(f"Number of observations used: {adf_res[3]}")
print(f"Critical Values: {adf_res[4]}")

if adf_res[1] <= 0.05:
    print("Reject the null hypothesis")
    print("The data is stationary")

else:
    print("The null hypothesis cannot be rejected")
    print("The data is not stationary")

**First Order Differencing**:

Since the p-value is greater than 0.05, we fail to reject the null hypothesis, which means the data is not stationary.

Lets do the first order differencing of the data to remove the varying Mean component. Below are the different ways that can be used to remove the varying mean and variance components.

**Differencing**: Subtracting the current observation from the previous one to achieve stationarity.
**Detrending**: Another technique for removing seasonality from time series data.
**Seasonal decomposition**: Decomposing the time series into seasonal, trend, and residual components.
**Log transformation**: Taking the natural logarithm of the data to stabilize variance.
**Log difference**: Combining log transformation and differencing to address both trend and seasonality.

In [ ]:
# Seasonal adjustment
df['Seasonal_Adjusted'] = df['Close'].diff(periods=7).dropna() ## An integer that controls the number of periods over which the difference is calculated.

# Differencing
df['Differenced'] = df['Close'].diff().dropna()

# Log Differencing
df['Log_Differenced'] = np.log(df['Close']/df['Close'].shift(1)).dropna()

df.head(10)

**Plot the data**

Validate the difference between actual, differenced, seasonal, and log differenced data.

In [ ]:
# Visualize the results
plt.figure(figsize=(12, 8))

# Original time series
plt.subplot(2, 2, 1)
plt.plot(final_data['Close'], color = 'blue')
plt.title('Nifty 50 - Original Data')
plt.xlabel('Year')
plt.ylabel('Closing Value')

# Seasonal Adjusted
plt.subplot(2, 2, 2)
plt.plot(df['Seasonal_Adjusted'], color = 'orange')
plt.title('Seasonal Adjusted Time Series')
plt.xlabel('Year')
plt.ylabel('Seasonal Adjusted Closing Value')

# Differenced
plt.subplot(2, 2, 3)
plt.plot(df['Differenced'], color = 'green')
plt.title('Differenced Time Series')
plt.xlabel('Year')
plt.ylabel('Differenced Closing Value')

# Log Differenced
plt.subplot(2, 2, 4)
plt.plot(df['Log_Differenced'], color = 'red')
plt.title('Log Differenced Time Series')
plt.xlabel('Year')
plt.ylabel('Log Differenced Closing Value')

plt.tight_layout()
plt.show()

**Interpretation of HeatMap**:

1. Seasonal Patterns
2. Increasing Trend Over Time
3. Anomalies and Deviations: 2020 shows an anamoly in both the Heat Maps

In [ ]:
pivot_date = df[['Year','Month','Close']]
pivot_diff = df[['Year','Month','Differenced']]
pivot_log = df[['Year','Month','Log_Differenced']]

df_pivot = pivot_date.pivot_table(index="Month", columns='Year', values='Close', aggfunc='mean')
df_diff_pivot = pivot_diff.pivot_table(index="Month", columns='Year', values='Differenced', aggfunc='mean')

# df_pivot.head()

plt.figure(figsize=(12, 6))

# Plotting the heatmap
plt.subplot(1,2,1)
sns.heatmap(df_pivot, cmap='YlGnBu', annot=False)
plt.title("Nifty 50 - Closing Value (2015 - 2023)")

plt.subplot(1,2,2)
sns.heatmap(df_diff_pivot, cmap='YlGnBu', annot=False)
plt.title("Nifty 50 - Differenced Value (2015 - 2023)")
plt.show()


**Stationary check post differencing**.

The differenced data has approximately more or less constant mean when we apply windows.

But, lets calculate the ADF value again to re-confirm

In [ ]:
# Perform the ADF test

adf_res_mod = adfuller(df['Differenced'].dropna(), autolag='AIC')
print(f"ADF Statistic: {adf_res_mod[0]}")
print(f"p-value: {adf_res_mod[1]}")
print(f"#Lags Used: {adf_res_mod[2]}")
print(f"Number of Observations Used: {adf_res_mod[3]}")
print(f"Critical Values: {adf_res_mod[4]}")

if adf_res_mod[1] <= 0.05:
    print("Reject the null hypothesis")
    print("The data is stationary")

else:
    print("The null hypothesis cannot be rejected")
    print("The data is not stationary")


The p-value obtained is less than the significance level of 0.05, and the ADF statistic is less than any of the critical values.
we reject the Null hypothesis in favor of the alternative. So, the time series is, in fact, stationary

**Histogram** representation of actual data and its differenced form.
Differenced data is approximately following Guassian Distribution

Variance plot as well for the differenced and log differenced data. Is the differenced data stationary? Though the differenced data follows Gaussian distribution, we see changes in the variance plot.


In [ ]:
plt.figure(figsize=(12, 5))

## Histogram of the Original data

plt.subplot(1,2,1)
final_data['Close'].dropna().hist()
plt.title("Original Data")

## Histogram of the Differenced data

plt.subplot(1,2,2)
df['Differenced'].dropna().hist()
plt.title("Differenced Data")
plt.show()


In [ ]:
## Now lets check how is the variance. Is the differenced data stationary or not
plt.figure(figsize=(12, 6))
# add the 20 day rolling standard deviation:
df['Differenced'].dropna().rolling(window=20).std().plot(style='r')
df['Log_Differenced'].dropna().rolling(window=20).std().plot(style='b')
# So the Variance changing over time.


**ACF and PACF** Plots

Lets check ACF and PACF details of the differenced data for 20 lags

In [ ]:
# ACF and PACF of the differenced series
plt.figure(figsize=(12, 6))

plot_acf(df['Differenced'].dropna(), lags=20)

plot_pacf(df['Differenced'].dropna(), lags=20)
plt.show()

**Based on the ACF and PACF plots, it shows complete white noise which need to be modelled.**

Let's check the PSD of the data to understand the frequency domain details of its respective time domain.

**Power Spectral Density** - **Welch Method** using **Hann** window.


In [ ]:
# PSD of the Differenced timeseries
f,PSD = signal.welch(df['Close'],fs=len(df['Differenced'].dropna()), window='hann')
plt.semilogy(f,PSD)
plt.show()

# PSD shows varying frequency details

Get the train and test datasets

In [ ]:
# split into train and test sets
X = df['Close'].dropna().values
Y = df_test['Close'].dropna().values
train, test = X, Y

**Random walk with drift**

The ACF and PACF happens to show pure white noise.

If after differencing your time series, both the ACF and PACF plots indicate only white noise, then the most likely model to identify is a simple ARIMA(0,1,0) model, which essentially represents a "random walk with drift" where the only significant component is the first difference of the data, meaning the change between consecutive observations is essentially random noise

ARIMA(0,1,0):
"0" for AR: No autoregressive component (no dependence on past values).
"1" for I: One order of differencing is required to achieve stationarity.
"0" for MA: No moving average component.

In [ ]:
# Fit ARMA(0,1,0) model
model = ARIMA(df['Close'].dropna(), order=(0,1,0), freq = "D")
fitted_model = model.fit()
print(fitted_model.summary())
step = len(df_test['Close'].dropna())

# Forecast next length of the test data steps
forecast = fitted_model.get_forecast(steps=step)
forecast_summary = forecast.summary_frame()

residuals = pd.DataFrame(fitted_model.resid, columns=["Values"])
# summary stats of residuals
print(residuals.describe())


**Plot** the Residuals of Random Walk with Drift

One of the most important steps in ARIMA modeling is to check if the residual series that is generated is **stationary**. Residuals are the difference between observed values and those produced by a model. It help us to determine whether a linear model is appropriate in modeling the given data

In [ ]:
# line plot of residuals
residuals.plot()
plt.title("Line Plot of Residuals.")

# density plot of residuals
residuals.plot(kind='kde')
plt.title("Density Plot of Residuals.")
plt.show()

In [ ]:
forecast_vals = fitted_model.forecast(steps=step)

# Set forecast index to a date range for 2024
forecast_summary.index = pd.date_range(start='2024-01-01', periods=step, freq='D')

# Plot the forecasted values against the actual test data

plt.figure(figsize=(10,6))
plt.plot(df['Close'].dropna().index, df['Close'], label='Training Data', color='blue')
plt.plot(df_test['Close'].index, df_test['Close'], label='Actual 2024 Data', color='red')
plt.plot(forecast_summary.index, forecast_vals, label='Forecasted 2024 Data', color='green', linestyle='--')

#plt.plot(np.arange(len(df['Close']), len(df['Close']) + 300), forecast.predicted_mean, label="Forecast", color='orange')

plt.title("ARMA(0,1,0) Model Forecast")
plt.legend()
plt.show()

df.head()

Checking MSE, RMSE, R^2, and MAPE

In [ ]:
# Evaluate the model using error metrics
# Ensure both series have the same length and index before calculating R^2

# Reset index of df_test['Close'] to match forecast_vals index
y_true = df_test['Close'].dropna().reset_index(drop=True)
y_pred = pd.Series(forecast_vals).reset_index(drop=True)

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Calculate R^2 using the aligned series
r2 = r2_score(y_true, y_pred)
print(f'R-squared (R^2): {r2}')

# Calculate MAPE
masked_actual = np.ma.masked_array(y_true, mask=y_true==0) #Mask values where y_true is 0 to avoid division by zero.
MAPE = (np.fabs(masked_actual.filled(0) - y_pred.to_numpy())/masked_actual.filled(1)).mean() #Convert masked_actual and y_pred to NumPy arrays for compatibility.

print(f'MAPE: {MAPE}')

Lets choose an appropriate model by identifying AIC values for a set of models that include variations of AR, MA, ARMA, and ARIMA

Checking AIC Values for various models. Since auto_arima is ignoring a few models, we are manually geenrating AIC for sample models.

In [ ]:
# Suppress warnings
warnings.filterwarnings("ignore") # Suppress all warnings

# Create a list to store the models and their AIC values
models = []

# Iterate through p, d, and q values up to the specified limits
for p in range(4):  # Iterate up to p=3
    for d in range(3): # Iterate up to d=2
        for q in range(4): # Iterate up to q=3
            try:
                model = ARIMA(df['Close'].dropna(), order=(p,d,q))
                results = model.fit()
                models.append((p, d, q, results.aic))
                print(f"ARIMA({p},{d},{q}): AIC={results.aic}") # Print the result of each iteration
            except Exception as e:
                print(f"Error fitting ARIMA({p},{d},{q}): {e}") # print the exception if there is one

# Find the model with the lowest AIC
best_model = min(models, key=lambda x: x[3])

print(f"Best Model: ARIMA({best_model[0]},{best_model[1]},{best_model[2]}) with AIC={best_model[3]}")


**ARIMA Modelling**:

As observed AIC value is less for ARIMA Order (ignoring seasonality) -  (1 2 2) - 36137.73064054128.
So lets try to fit 1,2,2 and check the output.

In [ ]:
# Forecast next steps based on the length of the forecast
train_data = df['Close'].dropna()
test_data = df_test['Close']
forecast_steps = len(test_data)

model = ARIMA(train_data, order=(best_model[0],best_model[1],best_model[2]),freq="D")
fitted_model = model.fit()
forecast = fitted_model.get_forecast(steps=forecast_steps,freq="D")
forecast_summary = forecast.summary_frame()

forecast_vals = fitted_model.forecast(steps=forecast_steps)

# Set forecast index to a date range for 2024
forecast_summary.index = pd.date_range(start='2024-01-01', periods=forecast_steps, freq='D')

# Plot the forecasted values against the actual test data

plt.figure(figsize=(10,6))
# plt.plot(train_data.index, train_data, label='Training Data', color='blue')
plt.plot(test_data.index, test_data, label='Actual 2024 Data', color='red')
plt.plot(forecast_summary.index, forecast_vals, label='Forecasted 2024 Data', color='green', linestyle='--')
plt.title('NSE Data: Actual vs Forecasted for 2024')
plt.legend()
plt.show()

df.head()

**MSE, RMSE, R^2 and MAPE**

In [ ]:
# Evaluate the model using error metrics
# Ensure both series have the same length and index before calculating R^2

# Reset index of df_test['Close'] to match forecast_vals index
y_true = df_test['Close'].dropna().reset_index(drop=True)
y_pred = pd.Series(forecast_vals).reset_index(drop=True)

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Calculate R^2 using the aligned series

r2 = r2_score(y_true, y_pred)
print(f'R-squared (R^2): {r2}')

# Calculate MAPE
masked_actual = np.ma.masked_array(y_true, mask=y_true==0) #Mask values where y_true is 0 to avoid division by zero.
MAPE = (np.fabs(masked_actual.filled(0) - y_pred.to_numpy())/masked_actual.filled(1)).mean() #Convert masked_actual and y_pred to NumPy arrays for compatibility.

print(f'MAPE: {MAPE}')

Trying **SARIMAX** -
As the ARIMA has forecasted the trend accurately but the seasonality component is missing, checking the minimum AIC value for various model orders and seasonal orders.

In [ ]:
model = pm.auto_arima(train_data, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=7,              # frequency of series (weekly)
                      d=2,           # let model determine 'd'
                      seasonal=True,   # Seasonality
                      start_P=0,
                      D=1,
                      trace=True,
                      error_action='ignore',
                      suppress_warnings=True,
                      stepwise=True)

print(model.summary())

Per Auto ARIMA ***among the given set of models***, with seasonality, the best model reported was **SARIMAX(3,2,0)(2,1,0)[7]**
Let's try to model the data using this order and seasonality.

In [ ]:
seasonal_order = (2, 1, 0, 7)  # (seasonality for daily data)
model = SARIMAX(df['Close'].dropna(), order=(3, 2, 0), seasonal_order=seasonal_order)
model_fit = model.fit()

# Forecast the data for 2024 (forecasting steps = length of test data)
forecast_steps = len(df_test['Close'].dropna())
forecast_Values = model_fit.forecast(steps=forecast_steps)

# Plot the forecasted values against the actual test data

plt.figure(figsize=(10,6))
#plt.plot(df.index, train_data, label='Training Data', color='blue')
plt.plot(df_test.index, test_data, label='Actual 2024 Data', color='red')
plt.plot(forecast_summary.index, forecast_Values, label='Forecasted 2024 Data', color='green', linestyle='--')
plt.title('NSE Data: Actual vs Forecasted for 2024')
plt.legend()
plt.show()

df.head()

**MSE, RMSE, R^2, MAPE**

In [ ]:
# Evaluate the model using error metrics
# Ensure both series have the same length and index before calculating R^2

# Reset index of df_test['Close'] to match forecast_vals index
y_true = df_test['Close'].dropna().reset_index(drop=True)
y_pred = pd.Series(forecast_vals).reset_index(drop=True)

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Calculate R^2 using the aligned series

r2 = r2_score(y_true, y_pred)
print(f'R-squared (R^2): {r2}')

# Calculate MAPE
masked_actual = np.ma.masked_array(y_true, mask=y_true==0) #Mask values where y_true is 0 to avoid division by zero.
MAPE = (np.fabs(masked_actual.filled(0) - y_pred.to_numpy())/masked_actual.filled(1)).mean() #Convert masked_actual and y_pred to NumPy arrays for compatibility.

print(f'MAPE: {MAPE}')

**Why Non-Linear model??**

Though we differenced the data and assumed the process to be stationary, as we have seen that there is still varying component of the Variance with absolute time. This indicates **Heteroscedasticity**. So, lets try Non-linear model to fit the data.

**ARCH** is a statistical model used to **analyze and forecast the *volatility* of time series data**, particularly in financial markets. It's based on the idea that the variance of a time series is not constant but rather depends on past values of the series and its own past variances.

1. The time series data should be stationary before applying ARCH. Hence, modelling the differenced output.
2. p - The model order of the ARCH determines how many past squared errors need to be included.

**Plot** the ARCH model fit Vs Train and Test data.

In [ ]:
train_data,test_data = df['Close'],df_test['Close']

# Fit a simple ARCH model to get residuals
basic_arch_model = arch_model(train_data, vol='ARCH', p=1).fit(disp='off')
residuals = basic_arch_model.resid

# Check for normality (e.g., using Shapiro-Wilk test)
shapiro_test = stats.shapiro(residuals)
print(f"Shapiro-Wilk Test p-value: {shapiro_test.pvalue}")

# If not normal, consider Box-Cox transformation
if shapiro_test.pvalue < 0.05:
    transformed_data, lambda_value = boxcox(train_data)  # Apply Box-Cox
    print(f"Box-Cox Transformation Lambda: {lambda_value}")
    # Proceed with modeling using 'transformed_data'
else:
    transformed_data = train_data  # No transformation needed



In [ ]:
# Suppress warnings
warnings.filterwarnings("ignore") # Suppress all warnings

# Define parameter grid for 'p' (and potentially other parameters)
param_grid = {'p': [1, 2, 3, 4, 5]}

# Use TimeSeriesSplit for cross-validation with time series data
tscv = TimeSeriesSplit(n_splits=5)

# Iterate through models and evaluate
best_model = None
best_score = float('inf')  # Initialize with a large value

for p_value in param_grid['p']:
    model = arch_model(transformed_data, vol='ARCH', p=p_value)
    scores = []
    for train_index, val_index in tscv.split(transformed_data):
        train_fold, val_fold = transformed_data[train_index], transformed_data[val_index]
        fold_model = model.fit(disp='off', last_obs=len(train_fold))  # Fit on train fold
        forecast = fold_model.forecast(horizon=len(val_fold), reindex=False)
        # Calculate a score (e.g., RMSE) on the validation fold
        score = np.sqrt(mean_squared_error(val_fold, forecast.variance.values[-1, :]))
        scores.append(score)
    avg_score = np.mean(scores)

    if avg_score < best_score:
        best_score = avg_score
        best_model = model
        best_p = p_value

print(f"Best p value: {best_p}, Best Score (RMSE): {best_score}")

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/uni

In [ ]:
# Fit a simple ARCH model to get residuals
basic_arch_model = arch_model(transformed_data, vol='ARCH', p=4).fit(disp='off')
# Fit the model
model_fit = model.fit()

# Print model summary
print(model_fit.summary())

# Forecast the next 10 days of volatility
forecast = model_fit.forecast(horizon=10)

# Extract the forecasted volatility (standard deviation)
forecasted_volatility = forecast.variance[-1:] ** 0.5

# Plot the forecasted volatility
plt.figure(figsize=(10, 6))
plt.plot(forecasted_volatility.T)
plt.title(f"Forecasted Volatility for the Next 10 Days")
plt.xlabel('Days')
plt.ylabel('Volatility')
plt.show()

In [ ]:
# GARCH(1, 1) model
garch_model = arch_model(transformed_data, vol='GARCH', p=1, q=1).fit(disp='off')

# EGARCH(1, 1) model
egarch_model = arch_model(transformed_data, vol='EGARCH', p=1, o=1, q=1).fit(disp='off')

# ... (compare performance using metrics and visualization)

In [ ]:
# Calculate rolling window variance of test data
window_size = 20  # Adjust as needed
rolling_variance = pd.Series(test_data).rolling(window=window_size).var().dropna()

# Get predicted volatility from the best model
forecast = best_model.forecast(horizon=len(test_data), reindex=False)
predicted_volatility = forecast.variance.values[-1, :]  # Extract volatility

# Plot
plt.figure(figsize=(12, 6))
plt.plot(rolling_variance.index, rolling_variance, label='Actual Rolling Variance', color='blue')
plt.plot(rolling_variance.index, predicted_volatility[:len(rolling_variance)], label='Predicted Volatility', color='red', linestyle='--')
plt.title('Predicted Volatility vs. Actual Rolling Variance')
plt.legend()
plt.show()